## Cell 1: Environment Setup and Data Loading
This cell initializes the workspace by importing necessary libraries and loading the raw data from your Excel file.

* **Libraries:** We use pandas for data manipulation and numpy for conditional logic.

* **Data Sources:** It loads the main interaction database (LRP-IntDB) and the Detection Techniques dictionary (DT), which contains the classification for scoring.

In [37]:
import pandas as pd
import numpy as np

# File path definition
file_path = 'LRP-IntDB (30).xlsx'

# Load the full Excel file
xl = pd.ExcelFile(file_path)

# Load the main database sheet
df_main = xl.parse('LRP-IntDB')

# Load the Detection Technique (DT) dictionary
# Header=1 is used because the DT sheet has a specific metadata structure in the first row
df_dt = xl.parse('DT', header=1) 

print(f"✅ Database loaded: {len(df_main)} rows.")
print(f"✅ Detection Technique (DT) dictionary loaded.")

✅ Database loaded: 623 rows.
✅ Detection Technique (DT) dictionary loaded.


## Cell 2: Building the Technique Mapping
To calculate scores, we need to know if a technique is Genomic, Protein, or Cell-based. This cell creates a "lookup dictionary" from the Excel sheet.

* **Logic:** It iterates through the columns of the DT sheet and maps every technique name to its respective category.

* **Purpose:** This allows the script to instantly categorize any technique found in the main database.

In [38]:
# Create a mapping: Technique Name -> Category (Group)
tech_to_group = {}

# Define dictionary columns based on the Excel structure
columnas_grupos = {
    'Genomic Experiments': 'Genomic',
    'Protein Experiments': 'Protein',
    'Cell Experiments': 'Cell'
}

for col_excel, grupo_id in columnas_grupos.items():
    if col_excel in df_dt.columns:
        # Extract techniques, remove nulls and trim whitespace
        tecnicas = df_dt[col_excel].dropna().unique()
        for t in tecnicas:
            tech_to_group[t.strip()] = grupo_id

print(f"📊 Dictionary built: {len(tech_to_group)} classified techniques.")

📊 Dictionary built: 35 classified techniques.


## Cell 3: Data Pre-processing and Expansion
Before scoring, we must clean the data and handle rows that contain multiple techniques separated by commas.

* **ProteinID:** We create a unique identifier. If a Uniprot ID is missing (*), we generate a temporary ID using the protein's name.

* **Explode Logic:** Many rows list techniques like "Western Blot, ELISA". This cell splits them into individual rows so each experiment is counted correctly.

In [39]:
# 1. Create a consistent unique ID for proteins
df_main['ProteinID'] = np.where(
    (df_main['ID Uniprot A'] != '*') & (df_main['ID Uniprot A'].notna()), 
    df_main['ID Uniprot A'], 
    "NAME_" + df_main['Protein Name A'].str.replace(' ', '_').str.replace(':', '')
)

# 2. Clean 'Cell Type' column
df_main['Cell Type'] = df_main['Cell Type'].fillna('*').astype(str).str.strip()

# 3. Technique Expansion (Explode)
# Splits comma-separated strings into individual rows
df_expanded = df_main.assign(
    Technique=df_main['Detection technique'].str.split(',')
).explode('Technique')

df_expanded['Technique'] = df_expanded['Technique'].str.strip()

print(f"📈 Data pre-processing complete.")

📈 Data pre-processing complete.


## Cell 4: The Scoring Algorithm (The "Golden Rule")
This is the core of the script. It calculates the interaction confidence score based on experimental evidence and inheritance.

* **Scoring Weights: Binary** techniques receive a flat $0.35$. **HTS** (High-Throughput) techniques start at $0.25$ and decrease by $0.05$ for every additional experiment in the same category to prevent "easy" inflation.

* **Inheritance Logic:**

    1. **Triple Star ( * , * ,*):** Pure global evidence. These rows calculate their own score.

    2. **Has Organism:** Rows with no Cell Type but specific Organism data. They inherit evidence from the Triple Star.

    3. **Specific Cell Type:** Rows with a named cell (e.g., "Hepatocyte"). They inherit evidence from the Triple Star.
    
* **Normalization:** The final sum is divided by $1.1$ and capped at $1.0$.

In [40]:
def calculate_group_score(rows):
    BINARY_WEIGHT = 0.35
    HTS_BASE_WEIGHT = 0.25
    HTS_REDUCTION_STEP = 0.05
    MIN_HTS_WEIGHT = 0.05
    total_score = 0
    group_counts = {'Genomic': 0, 'Protein': 0, 'Cell': 0}
    
    for _, row in rows.iterrows():
        tech = str(row['Technique'])
        td_type = str(row['TD Type'])
        category = tech_to_group.get(tech, 'Protein') 
        
        if 'Binary' in td_type:
            weight = BINARY_WEIGHT
        else:
            count = group_counts.get(category, 0)
            weight = max(MIN_HTS_WEIGHT, HTS_BASE_WEIGHT - (count * HTS_REDUCTION_STEP))
            group_counts[category] += 1
        total_score += weight
    
    return min(total_score / 1.1, 1.0)

# 1. Clean columns for grouping
for col in ['Cell Type', 'Organism', 'Cell line/ Tissue']:
    df_expanded[col] = df_expanded[col].fillna('*').astype(str).str.strip()

# 2. Group Labeling Logic
def get_final_group_label(row):
    p_id = row['ProteinID']
    c_type = row['Cell Type']
    org = row['Organism']
    c_line = row['Cell line/ Tissue']
    
    if c_type == '*' and org == '*' and c_line == '*':
        return f"{p_id}_TRIPLE_STAR"
    if c_type == '*' and org != '*':
        return f"{p_id}_HAS_ORGANISM"
    return f"{p_id}_{c_type}"

df_expanded['Group_Label'] = df_expanded.apply(get_final_group_label, axis=1)

# 3. Calculate Scores per Group
final_results = []
unique_labels = df_expanded['Group_Label'].unique()

for label in unique_labels:
    specific_rows = df_expanded[df_expanded['Group_Label'] == label]
    p_id = specific_rows.iloc[0]['ProteinID']
    
    # Non-global groups inherit evidence from the 'TRIPLE_STAR' group
    if "_TRIPLE_STAR" not in label:
        wildcards = df_expanded[df_expanded['Group_Label'] == f"{p_id}_TRIPLE_STAR"]
        evidence_pool = pd.concat([specific_rows, wildcards]).drop_duplicates()
    else:
        evidence_pool = specific_rows
        
    score = calculate_group_score(evidence_pool)
    final_results.append({'Group_Label': label, 'Score': score})

# 4. Final Integration
scores_map = pd.DataFrame(final_results)
df_final = df_main.copy()
for col in ['Cell Type', 'Organism', 'Cell line/ Tissue']:
    df_final[col] = df_final[col].fillna('*').astype(str).str.strip()

df_final['Group_Label'] = df_final.apply(get_final_group_label, axis=1)
df_final = df_final.merge(scores_map, on='Group_Label', how='left').drop(columns=['Group_Label'])

print(f"✅ Scoring process completed.")

✅ Scoring process completed.


## Cell 5: Exporting the Final Dataset
Finally, we clean up the temporary columns used for calculation and save the result into a new Excel file.

In [41]:
# 1. Create a safe copy
df_to_save = df_final.copy()

# 2. Remove the temporary ProteinID column to restore the original 'ID Uniprot A' look
if 'ProteinID' in df_to_save.columns:
    df_to_save = df_to_save.drop(columns=['ProteinID'])

# 3. Save to Excel
output_file = 'LRP-IntDB_Final_Scored.xlsx'
df_to_save.to_excel(output_file, index=False)

print(f"🎉 Final file saved as: {output_file}")

🎉 Final file saved as: LRP-IntDB_Final_Scored.xlsx
